In [5]:
import cv2
from ultralytics import YOLO


# Carregar modelo leve (você pode trocar por outro)
model = YOLO('yolov8s.pt')

# Primeira tentativa rodando o Yolo

In [ ]:
import cv2
from ultralytics import YOLO

# Carrega o modelo
model = YOLO('yolov8s.pt')

# Define classes relevantes à navegação
CLASSES_UTEIS = ['person', 'bench', 'bicycle', 'car', 'chair', 'stop sign']

# Abre o vídeo (Etapa 1)
video_path = 'video_certo_edat.mp4'
cap = cv2.VideoCapture(video_path)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detecção
    results = model(frame, verbose=False)
    frame_copy = frame.copy()

    for result in results:
        for box in result.boxes:
            cls = model.names[int(box.cls)]
            conf = float(box.conf)

            # Só desenha se estiver na lista de interesse
            if cls in CLASSES_UTEIS and conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{cls} ({conf:.2f})"
                cv2.rectangle(frame_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_copy, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv2.imshow("Detecção de Objetos Relevantes", frame_copy)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Com o ROI

In [9]:
import cv2
from ultralytics import YOLO

model = YOLO('yolov8s.pt')

# Classes úteis
CLASSES_UTEIS = ['person', 'bench', 'bicycle', 'car', 'chair', 'stop sign']

video_path = 'praca_edat.mp4'
cap = cv2.VideoCapture(video_path)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape

# Define ROI: região central (50% da largura e altura)
    x1_roi = int(width * 0.10)
    x2_roi = int(width * 0.70)
    y1_roi = int(height * 0.35)
    y2_roi = int(height * 0.75)


    # Desenha ROI (só pra visualizar)
    cv2.rectangle(frame, (x1_roi, y1_roi), (x2_roi, y2_roi), (255, 255, 0), 2)

    results = model(frame, verbose=False)

    for result in results:
        for box in result.boxes:
            cls = model.names[int(box.cls)]
            conf = float(box.conf)
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            # Centro da caixa detectada
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            # Só mostra objetos dentro da ROI
            if (cls in CLASSES_UTEIS) and (x1_roi < cx < x2_roi) and (y1_roi < cy < y2_roi) and conf > 0.5:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{cls} ({conf:.2f})", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv2.imshow("Detecção com ROI - Zona de Interesse Frontal", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## Etapa 3

In [12]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO

############################################
# 1) MODELOS: YOLO (detecção) + MiDaS (profundidade)
############################################

# YOLO – use o mesmo que você já calibrou
yolo = YOLO('yolov8s.pt')

# MiDaS pequeno (mais leve pra CPU) – via torch.hub
device = torch.device('cpu')
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small', trust_repo=True).to(device).eval()
transform = torch.hub.load('intel-isl/MiDaS', 'transforms', trust_repo=True).small_transform

############################################
# 2) CONFIGS DO PROJETO
############################################

# Suas classes alvo (mapeadas para COCO)
CLASSES_UTEIS = [
    'person', 'bicycle', 'car', 'bench',
    'chair', 'stop sign', 'potted plant'  # aproxima lixeira em alguns contextos
]

# ROI – sua versão levemente à esquerda e central-baixa
def get_roi(frame):
    h, w = frame.shape[:2]
    x1 = int(w * 0.10)
    x2 = int(w * 0.70)
    y1 = int(h * 0.35)
    y2 = int(h * 0.75)
    return (x1, y1, x2, y2)

# Calibração (escala) da profundidade:
# MiDaS produz "inverso de profundidade" arbitrário. 
# Para converter para "metros aproximados", defina um ponto de referência no seu vídeo:
#   – meça uma distância real (ex.: um poste a ~5m) e ajuste abaixo até bater.
# Use um valor inicial (heurístico) e ajuste no relatório.
METER_SCALE = 4.0   # fator de escala (ajuste empírico no seu vídeo)

# Thresholds para risco (ajuste fino no seu vídeo)
DIST_DANGER_M = 2.5    # abaixo disso, já é “perto”
FLOW_APPROACH_MIN = 0.6 # velocidade relativa média (px/frame) para considerar aproximação
CENTER_BONUS = 0.15     # bônus de risco se bbox estiver no centro da ROI

############################################
# 3) Funções utilitárias
############################################

def predict_depth(frame_bgr):
    """Gera mapa de profundidade (inverso) com MiDaS + filtros (median + bilateral)."""
    img_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    inp = transform(img_rgb).to(device)
    with torch.no_grad():
        pred = midas(inp)
        pred = torch.nn.functional.interpolate(
            pred.unsqueeze(1),
            size=img_rgb.shape[:2],
            mode='bicubic',
            align_corners=False
        ).squeeze().cpu().numpy()
    # normaliza para [0,1] (inverso de profundidade)
    pred_norm = (pred - pred.min()) / (pred.max() - pred.min() + 1e-6)
    # pós-processamento: suavização
    pred_norm = cv2.medianBlur((pred_norm*255).astype(np.uint8), 5)
    pred_norm = cv2.bilateralFilter(pred_norm, d=7, sigmaColor=50, sigmaSpace=7)
    pred_norm = pred_norm.astype(np.float32) / 255.0
    return pred_norm  # 0..1 (maior = mais perto)

def invdepth_to_meters(invdepth):
    """Converte inverso de profundidade normalizado para distância aproximada em metros."""
    inv = float(invdepth)
    inv = max(inv, 1e-3)  # evita divisão por zero
    # distância ~ (constante / invdepth)
    meters = METER_SCALE / inv
    return meters

def bbox_center(b):
    x1, y1, x2, y2 = b
    return (int((x1+x2)//2), int((y1+y2)//2))

def iou(a, b):
    ax1, ay1, ax2, ay2 = a; bx1, by1, bx2, by2 = b
    inter_x1, inter_y1 = max(ax1, bx1), max(ay1, by1)
    inter_x2, inter_y2 = min(ax2, bx2), min(ay2, by2)
    iw, ih = max(0, inter_x2 - inter_x1), max(0, inter_y2 - inter_y1)
    inter = iw * ih
    area_a = (ax2-ax1)*(ay2-ay1); area_b = (bx2-bx1)*(by2-by1)
    union = area_a + area_b - inter + 1e-6
    return inter/union

def mean_flow_in_bbox(flow, bbox):
    x1, y1, x2, y2 = bbox
    x1, y1 = max(0, x1), max(0, y1)
    fx = flow[y1:y2, x1:x2, 0]
    fy = flow[y1:y2, x1:x2, 1]
    if fx.size == 0: 
        return 0.0, 0.0, 0.0
    mag = np.sqrt(fx*fx + fy*fy)
    return float(np.mean(fx)), float(np.mean(fy)), float(np.mean(mag))

def center_weight_in_roi(cx, cy, roi):
    x1, y1, x2, y2 = roi
    rx, ry = (x1+x2)/2, (y1+y2)/2
    rw, rh = (x2-x1), (y2-y1)
    dx = abs(cx - rx)/ (rw/2 + 1e-6)
    dy = abs(cy - ry)/ (rh/2 + 1e-6)
    d = np.sqrt(dx*dx + dy*dy)  # 0 = centro, 1 = borda
    return max(0.0, 1.0 - min(1.0, d))  # peso 0..1 (1 = centro)

############################################
# 4) Loop do vídeo com ROI + profundidade + fluxo + risco
############################################

video_path = 'praca_edat.mp4'
cap = cv2.VideoCapture(video_path)

prev_gray = None
prev_boxes = []

while True:
    ok, frame = cap.read()
    if not ok:
        break

    h, w = frame.shape[:2]
    roi = get_roi(frame)

    # Desenha ROI
    cv2.rectangle(frame, (roi[0], roi[1]), (roi[2], roi[3]), (255, 255, 0), 2)

    # Profundidade do frame atual
    invdepth = predict_depth(frame)  # 0..1, maior = mais perto

    # Fluxo óptico (com o frame anterior)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if prev_gray is None:
        prev_gray = gray.copy()

    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray,
                                        None, pyr_scale=0.5, levels=3,
                                        winsize=21, iterations=3,
                                        poly_n=5, poly_sigma=1.2, flags=0)

    # YOLO detecção
    results = yolo(frame, verbose=False)

    # Seleciona objetos de interesse **dentro da ROI**
    candidates = []
    for r in results:
        for b in r.boxes:
            cls = yolo.names[int(b.cls)]
            conf = float(b.conf)
            if conf < 0.45 or cls not in CLASSES_UTEIS:
                continue
            x1, y1, x2, y2 = map(int, b.xyxy[0])
            cx, cy = bbox_center((x1, y1, x2, y2))
            if not (roi[0] < cx < roi[2] and roi[1] < cy < roi[3]):
                continue

            # Profundidade dentro do bbox (mediana é robusta)
            x1c, y1c = max(0, x1), max(0, y1)
            x2c, y2c = min(w, x2), min(h, y2)
            box_depth = invdepth[y1c:y2c, x1c:x2c]
            if box_depth.size == 0:
                continue
            inv_med = float(np.median(box_depth))
            dist_m = invdepth_to_meters(inv_med)

            # Fluxo médio (velocidade relativa)
            mx, my, mmag = mean_flow_in_bbox(flow, (x1c, y1c, x2c, y2c))
            # Heurística de aproximação: módulo + componente vertical em direção à câmera:
            approach = mmag + max(0.0, -my)  # se my negativo (vindo "pra baixo"), soma

            # Centralidade na ROI (0..1)
            cweight = center_weight_in_roi(cx, cy, roi)

            candidates.append({
                'bbox': (x1, y1, x2, y2),
                'cls': cls, 'conf': conf,
                'dist_m': dist_m,
                'flow_mag': mmag,
                'approach': approach,
                'center_w': cweight,
                'center': (cx, cy)
            })

    # Heurística de PRIORIDADE: menor distância + mais central + maior aproximação
    target = None
    if candidates:
        # score simples: pondera distância (inverso), centralidade e aproximação
        for c in candidates:
            inv_d = 1.0 / (c['dist_m'] + 1e-6)
            c['score'] = 1.5*inv_d + 1.0*c['center_w'] + 0.8*c['approach']
        target = max(candidates, key=lambda x: x['score'])

    # Desenha todos os candidatos e destaca o "narrado"
    for c in candidates:
        x1, y1, x2, y2 = c['bbox']
        color = (0, 255, 0)
        if target is not None and c is target:
            color = (0, 0, 255)  # vermelho = alvo narrado
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        txt = f"{c['cls']} {c['dist_m']:.1f}m  v~{c['flow_mag']:.2f}"
        cv2.putText(frame, txt, (x1, max(20, y1-8)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Regra de RISCO (alerta)
    alert_text = ""
    if target:
        near = target['dist_m'] <= DIST_DANGER_M
        fast = target['approach'] >= FLOW_APPROACH_MIN
        centered = target['center_w'] >= (1.0 - CENTER_BONUS)  # mais central

        if (near and fast) or (near and centered):
            alert_text = f"ALERTA: {target['cls']} a {target['dist_m']:.1f} m"
            cv2.putText(frame, alert_text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 3)

    # Mostra
    cv2.imshow("Etapa 3 - Profundidade, Velocidade e Risco", frame)
    prev_gray = gray.copy()

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\lilia/.cache\torch\hub\intel-isl_MiDaS_master
C:\Users\lilia\OneDrive\Estudo\Visao_Computacional\T3\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\lilia\OneDrive\Estudo\Visao_Computacional\T3\venv\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Loading weights:  None


C:\Users\lilia\OneDrive\Estudo\Visao_Computacional\T3\venv\Lib\site-packages\torch\hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/rwightman/gen-efficientnet-pytorch/zipball/master" to C:\Users\lilia/.cache\torch\hub\master.zip
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_lite3-b733e338.pth" to C:\Users\lilia/.cache\torch\hub\checkpoints\tf_effici

# Próxima Etapa - Narração

In [11]:
import sys
print(sys.executable)

C:\Users\lilia\OneDrive\Estudo\Visao_Computacional\T3\venv\Scripts\python.exe


In [1]:
import cv2
import torch
import numpy as np
import pyttsx3
import time
from ultralytics import YOLO

############################################
# 1) CONFIGURAÇÃO DOS MODELOS
############################################
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = YOLO('yolov8s.pt')

# Carrega modelo de profundidade MiDaS
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small', trust_repo=True).to(device).eval()
transform = torch.hub.load('intel-isl/MiDaS', 'transforms', trust_repo=True).small_transform

############################################
# 2) CONFIGURAÇÃO DE CLASSES E ROI
############################################
CLASSES_UTEIS = ['person', 'bench', 'bicycle', 'car', 'chair', 'stop sign']
METER_SCALE = 4.0  # fator aproximado pra converter profundidade

# ROI — ajustável
ROI_X1, ROI_X2 = 0.15, 0.75
ROI_Y1, ROI_Y2 = 0.35, 0.75

############################################
# 3) CONFIGURAÇÃO DO TTS
############################################
tts = pyttsx3.init()
tts.setProperty('rate', 170)  # velocidade da fala
tts.setProperty('voice', 'brazil')  # tenta selecionar voz em português (pode variar por SO)

# Controle de narração
ultima_fala = 0
INTERVALO_FALA = 4  # segundos entre mensagens

############################################
# 4) FUNÇÃO DE NARRAÇÃO
############################################
def narrar(mensagem):
    global ultima_fala
    agora = time.time()
    if agora - ultima_fala > INTERVALO_FALA:
        print("🔊", mensagem)
        tts.say(mensagem)
        tts.runAndWait()
        ultima_fala = agora

############################################
# 5) LOOP PRINCIPAL
############################################
cap = cv2.VideoCapture('praca_edat.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape

    # Define ROI
    x1_roi = int(width * ROI_X1)
    x2_roi = int(width * ROI_X2)
    y1_roi = int(height * ROI_Y1)
    y2_roi = int(height * ROI_Y2)
    cv2.rectangle(frame, (x1_roi, y1_roi), (x2_roi, y2_roi), (255, 255, 0), 2)

    ########################################
    # DETECÇÃO
    ########################################
    results = model(frame, verbose=False)

    # Depth map
    input_batch = transform(frame).to(device)
    with torch.no_grad():
        depth = midas(input_batch)
        depth = torch.nn.functional.interpolate(depth.unsqueeze(1),
                                                size=frame.shape[:2],
                                                mode='bilinear',
                                                align_corners=False).squeeze().cpu().numpy()

    alerta_critico = False
    for result in results:
        for box in result.boxes:
            cls = model.names[int(box.cls)]
            conf = float(box.conf)
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            if cls not in CLASSES_UTEIS or conf < 0.5:
                continue

            # Centro da detecção
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

            # Só considera dentro da ROI
            if not (x1_roi < cx < x2_roi and y1_roi < cy < y2_roi):
                continue

            # Estima distância
            obj_depth = np.median(depth[y1:y2, x1:x2])
            distancia_m = METER_SCALE / max(obj_depth, 0.1)

            # Exibe info na tela
            texto = f"{cls} {distancia_m:.1f}m"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, texto, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            ########################################
            # LÓGICA DE ALERTA E NARRAÇÃO
            ########################################
            if distancia_m < 2.0:  # obstáculo muito próximo
                alerta_critico = True
                narrar(f"Atenção: {cls} a {distancia_m:.1f} metros à frente.")
            elif distancia_m < 4.0:
                narrar(f"{cls} a {distancia_m:.1f} metros. Siga em frente.")

    # Mostra telemetria opcional
    fps = cap.get(cv2.CAP_PROP_FPS)
    cv2.putText(frame, f"FPS: {fps:.1f}", (20, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Percepção + Narração", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\lilia/.cache\torch\hub\intel-isl_MiDaS_master
C:\Users\lilia\OneDrive\Estudo\Visao_Computacional\T3\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\lilia\OneDrive\Estudo\Visao_Computacional\T3\venv\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Loading weights:  None


Using cache found in C:\Users\lilia/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master
Using cache found in C:\Users\lilia/.cache\torch\hub\intel-isl_MiDaS_master


🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: person a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção

In [10]:
import cv2
import torch
import numpy as np
import pyttsx3
import time
from ultralytics import YOLO

############################################
# 1) CONFIGURAÇÃO DOS MODELOS
############################################
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Modelo YOLOv8 — detecção de objetos
model = YOLO('yolov8s.pt')

# Modelo MiDaS — estimativa de profundidade monocular
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small', trust_repo=True).to(device).eval()
transform = torch.hub.load('intel-isl/MiDaS', 'transforms', trust_repo=True).small_transform

############################################
# 2) CONFIGURAÇÃO DE CLASSES E ROI
############################################
CLASSES_UTEIS = ['person', 'bench', 'bicycle', 'car', 'chair', 'stop sign']
METER_SCALE = 4.0  # fator aproximado para converter profundidade em metros

# ROI (Zona de Interesse Frontal)
ROI_X1, ROI_X2 = 0.15, 0.75
ROI_Y1, ROI_Y2 = 0.35, 0.75

############################################
# 3) CONFIGURAÇÃO DO TTS (NARRAÇÃO)
############################################
tts = pyttsx3.init(driverName='sapi5')  # para Windows
tts.setProperty('rate', 170)  # velocidade da fala

# Controle de rate-limit por classe
ultima_fala_por_classe = {}
INTERVALO_FALA = 2.0  # segundos entre mensagens da mesma classe


def narrar(mensagem, classe):
    """Faz a narração, respeitando intervalo mínimo por classe."""
    global ultima_fala_por_classe
    agora = time.time()
    ultima = ultima_fala_por_classe.get(classe, 0)
    if agora - ultima > INTERVALO_FALA:
        print("🔊", mensagem)
        tts.say(mensagem)
        tts.runAndWait()
        ultima_fala_por_classe[classe] = agora


############################################
# 4) LOOP PRINCIPAL
############################################
cap = cv2.VideoCapture('praca_edat.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape

    # Desenha ROI
    x1_roi = int(width * ROI_X1)
    x2_roi = int(width * ROI_X2)
    y1_roi = int(height * ROI_Y1)
    y2_roi = int(height * ROI_Y2)
    cv2.rectangle(frame, (x1_roi, y1_roi), (x2_roi, y2_roi), (255, 255, 0), 2)

    ########################################
    # DETECÇÃO DE OBJETOS (YOLO)
    ########################################
    results = model(frame, verbose=False)

    ########################################
    # PROFUNDIDADE (MiDaS)
    ########################################
    input_batch = transform(frame).to(device)
    with torch.no_grad():
        depth = midas(input_batch)
        depth = torch.nn.functional.interpolate(
            depth.unsqueeze(1),
            size=frame.shape[:2],
            mode='bilinear',
            align_corners=False
        ).squeeze().cpu().numpy()

    ########################################
    # ANÁLISE DE OBJETOS
    ########################################
    for result in results:
        for box in result.boxes:
            cls = model.names[int(box.cls)]
            conf = float(box.conf)
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            if cls not in CLASSES_UTEIS or conf < 0.5:
                continue

            # Centro da detecção
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

            # Verifica se está na ROI
            if not (x1_roi < cx < x2_roi and y1_roi < cy < y2_roi):
                continue

            # Estima distância
            obj_depth = np.median(depth[y1:y2, x1:x2])
            distancia_m = METER_SCALE / max(obj_depth, 0.1)

            # Exibe na tela
            texto = f"{cls} {distancia_m:.1f}m"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, texto, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            ########################################
            # NARRAÇÃO (opção A)
            ########################################
            if distancia_m < 2.0:
                narrar(f"Atenção: {cls} a {distancia_m:.1f} metros à frente.", cls)
            elif distancia_m < 4.0:
                narrar(f"{cls} a {distancia_m:.1f} metros. Siga em frente.", cls)

    ########################################
    # TELEMETRIA OPCIONAL (FPS)
    ########################################
    fps = cap.get(cv2.CAP_PROP_FPS)
    cv2.putText(frame, f"FPS: {fps:.1f}", (20, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Percepção + Narração Contínua", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\lilia/.cache\torch\hub\intel-isl_MiDaS_master


Loading weights:  None


Using cache found in C:\Users\lilia/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master
Using cache found in C:\Users\lilia/.cache\torch\hub\intel-isl_MiDaS_master


🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: person a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção

KeyboardInterrupt: 

In [11]:
import cv2
import torch
import numpy as np
import pyttsx3
import time
from ultralytics import YOLO

############################################
# 1) CONFIGURAÇÃO DOS MODELOS
############################################
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# YOLOv8 para detecção
model = YOLO('yolov8s.pt')

# MiDaS para profundidade
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small', trust_repo=True).to(device).eval()
transform = torch.hub.load('intel-isl/MiDaS', 'transforms', trust_repo=True).small_transform

############################################
# 2) CONFIGURAÇÃO DE CLASSES E ROI
############################################
CLASSES_UTEIS = ['person', 'bench', 'bicycle', 'car', 'chair', 'stop sign']
METER_SCALE = 4.0

ROI_X1, ROI_X2 = 0.15, 0.75
ROI_Y1, ROI_Y2 = 0.35, 0.75

############################################
# 3) CONFIGURAÇÃO DO TTS
############################################
tts = pyttsx3.init(driverName='sapi5')
tts.setProperty('rate', 170)

# Seleciona voz PT-BR
for v in tts.getProperty('voices'):
    if 'portuguese' in v.name.lower() or 'brazil' in v.name.lower():
        tts.setProperty('voice', v.id)
        print(f"✅ Voz selecionada: {v.name}")
        break

ultima_fala_por_classe = {}
INTERVALO_FALA = 2.0

def narrar(mensagem, classe):
    agora = time.time()
    ultima = ultima_fala_por_classe.get(classe, 0)
    if agora - ultima > INTERVALO_FALA:
        print("🔊", mensagem)
        tts.say(mensagem)
        tts.runAndWait()
        ultima_fala_por_classe[classe] = agora

############################################
# 4) LOOP PRINCIPAL
############################################
cap = cv2.VideoCapture('praca_edat.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape

    # ROI
    x1_roi = int(width * ROI_X1)
    x2_roi = int(width * ROI_X2)
    y1_roi = int(height * ROI_Y1)
    y2_roi = int(height * ROI_Y2)
    cv2.rectangle(frame, (x1_roi, y1_roi), (x2_roi, y2_roi), (255, 255, 0), 2)

    # YOLO detecção
    results = model(frame, verbose=False)

    # MiDaS profundidade
    input_batch = transform(frame).to(device)
    with torch.no_grad():
        depth = midas(input_batch)
        depth = torch.nn.functional.interpolate(
            depth.unsqueeze(1),
            size=frame.shape[:2],
            mode='bilinear',
            align_corners=False
        ).squeeze().cpu().numpy()

    for result in results:
        for box in result.boxes:
            cls = model.names[int(box.cls)]
            conf = float(box.conf)
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            if cls not in CLASSES_UTEIS or conf < 0.5:
                continue

            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            if not (x1_roi < cx < x2_roi and y1_roi < cy < y2_roi):
                continue

            # Direção (esquerda / direita / frente)
            largura_roi = x2_roi - x1_roi
            pos_relativa = (cx - x1_roi) / largura_roi
            if pos_relativa < 0.33:
                direcao = "à esquerda"
            elif pos_relativa > 0.66:
                direcao = "à direita"
            else:
                direcao = "à frente"

            # Distância estimada
            obj_depth = np.median(depth[y1:y2, x1:x2])
            distancia_m = METER_SCALE / max(obj_depth, 0.1)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            texto = f"{cls} {distancia_m:.1f}m {direcao}"
            cv2.putText(frame, texto, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            # Narração com direção
            if distancia_m < 2.0:
                narrar(f"Atenção: {cls} a {distancia_m:.1f} metros {direcao}.", cls)
            elif distancia_m < 4.0:
                narrar(f"{cls} a {distancia_m:.1f} metros {direcao}.", cls)

    # FPS
    fps = cap.get(cv2.CAP_PROP_FPS)
    cv2.putText(frame, f"FPS: {fps:.1f}", (20, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Percepção + Narração Direcional", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\lilia/.cache\torch\hub\intel-isl_MiDaS_master


Loading weights:  None


Using cache found in C:\Users\lilia/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master
Using cache found in C:\Users\lilia/.cache\torch\hub\intel-isl_MiDaS_master


✅ Voz selecionada: Microsoft Maria Desktop - Portuguese(Brazil)
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à esquerda.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: car a 0.0 metros à frente.
🔊 Atenção: person a 0.0 metros à esquerda.
🔊 Atenção: car a 0.0 metros à direita.
🔊 Atenção: car a 0.0 metros à direita.
🔊 Atenção: car a 0.0 metros à direita.
🔊 Atenção: car a 0.0 metros à direita.


KeyboardInterrupt: 

In [4]:
import pyttsx3

print("Inicializando TTS...")
tts = pyttsx3.init(driverName='sapi5')

print("\n=== VOZES DISPONÍVEIS ===")
for v in tts.getProperty('voices'):
    print(v.id, "-", v.name)

tts.setProperty('rate', 170)
tts.say("Teste de voz. Se você ouviu isto, o T T S está funcionando.")
tts.runAndWait()
print("✅ Fim do teste.")


Inicializando TTS...

=== VOZES DISPONÍVEIS ===
HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_PT-BR_MARIA_11.0 - Microsoft Maria Desktop - Portuguese(Brazil)
HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0 - Microsoft Zira Desktop - English (United States)
✅ Fim do teste.


In [6]:
!pip install gTTS playsound==1.2.2


   -------------------------- ------------- 2/3 [gTTS]
   ---------------------------------------- 3/3 [gTTS]



### Não está saindo a narração

In [12]:
##############################################
# ETAPA 4 — NARRAÇÃO & UX (funcional no Jupyter)
##############################################

from gtts import gTTS
from playsound import playsound
import tempfile
import time
import cv2
import torch
import numpy as np

# ===========================================
# CONFIGURAÇÃO DE NARRAÇÃO (TTS)
# ===========================================
ultima_narracao = 0
intervalo_minimo = 3.0  # segundos entre falas para evitar "spam"

def narrar(mensagem, prioridade=False):
    global ultima_narracao
    agora = time.time()

    # Se for alerta crítico, fala mesmo que o tempo mínimo não tenha passado
    if (agora - ultima_narracao) < intervalo_minimo and not prioridade:
        return

    print(f"🔊 {mensagem}")
    try:
        with tempfile.NamedTemporaryFile(delete=True, suffix=".mp3") as f:
            tts = gTTS(mensagem, lang='pt-br')
            tts.save(f.name)
            playsound(f.name)
        ultima_narracao = agora
    except Exception as e:
        print(f"[ERRO TTS] {e}")

# ===========================================
# FUNÇÕES AUXILIARES DE DETECÇÃO E PROFUNDIDADE
# ===========================================

def estimar_distancia(mask, depth_map):
    """Retorna a distância média (em escala relativa) de um objeto"""
    if mask is None or np.sum(mask) == 0:
        return None
    profundidades = depth_map[mask > 0]
    return float(np.median(profundidades))

def avaliar_risco(distancia, posicao_roi):
    """Heurística simples de risco"""
    if distancia is None:
        return "sem risco"
    if distancia < 0.3:
        return "⚠️ Colisão iminente"
    elif distancia < 0.6:
        return "🚶 Obstáculo próximo"
    elif posicao_roi == "frente":
        return "🟢 Caminho livre"
    else:
        return "seguro"

# ===========================================
# LOOP PRINCIPAL (exemplo)
# ===========================================

cap = cv2.VideoCapture("praca_edat.mp4")

fps_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Simula profundidade com valores aleatórios (substitua pela saída real do MiDaS)
    depth_map = np.random.rand(frame.shape[0], frame.shape[1])

    # Simula detecção de pessoa no centro
    mask = np.zeros_like(depth_map)
    h, w = depth_map.shape
    mask[h//3:2*h//3, w//3:2*w//3] = 1

    distancia = estimar_distancia(mask, depth_map)
    risco = avaliar_risco(distancia, "frente")

    # Exemplo de narração
    if "Colisão" in risco:
        narrar("Atenção! Obstáculo à frente!", prioridade=True)
    elif "próximo" in risco:
        narrar("Obstáculo à frente. Desvie à direita.")
    elif "livre" in risco:
        narrar("Caminho livre.", prioridade=False)

    # Telemetria na tela
    fps = 1.0 / (time.time() - fps_time)
    fps_time = time.time()
    cv2.putText(frame, f"{risco} - Distância: {distancia:.2f} - FPS: {fps:.1f}",
                (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    cv2.imshow("Detecção com TTS", frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


🔊 Obstáculo à frente. Desvie à direita.
[ERRO TTS] [Errno 13] Permission denied: 'C:\\Users\\lilia\\AppData\\Local\\Temp\\tmp9mcmk592.mp3'
🔊 Obstáculo à frente. Desvie à direita.
[ERRO TTS] [Errno 13] Permission denied: 'C:\\Users\\lilia\\AppData\\Local\\Temp\\tmpk2cbc5qu.mp3'
🔊 Obstáculo à frente. Desvie à direita.
[ERRO TTS] [Errno 13] Permission denied: 'C:\\Users\\lilia\\AppData\\Local\\Temp\\tmp6j84ojh2.mp3'
🔊 Obstáculo à frente. Desvie à direita.
[ERRO TTS] [Errno 13] Permission denied: 'C:\\Users\\lilia\\AppData\\Local\\Temp\\tmp6s2xh0n0.mp3'
🔊 Obstáculo à frente. Desvie à direita.
[ERRO TTS] [Errno 13] Permission denied: 'C:\\Users\\lilia\\AppData\\Local\\Temp\\tmpkpv8_tjk.mp3'
🔊 Obstáculo à frente. Desvie à direita.
[ERRO TTS] [Errno 13] Permission denied: 'C:\\Users\\lilia\\AppData\\Local\\Temp\\tmpqvibj46_.mp3'
🔊 Obstáculo à frente. Desvie à direita.
[ERRO TTS] [Errno 13] Permission denied: 'C:\\Users\\lilia\\AppData\\Local\\Temp\\tmp23liako5.mp3'
🔊 Obstáculo à frente. Desvi